In [1]:
#библиотеки

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as ps
import plotly.graph_objects as go
import scipy
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import statsmodels.api as sm


df = pd.read_csv("DataSet3_2.csv", sep=";")
#df = df.drop(columns=['pc'], axis=1)
df['Data'] = pd.to_datetime(df['Data'], format="%d.%m.%Y")
df['Seconds'] = df['Data'].apply(lambda x: int(x.timestamp()))
df['Dollar']=df['Dollar'].str.replace(',', '.').astype(float).round()
df['Oil Brent']=df['Oil Brent'].str.replace(',', '.').astype(float).round()
df.head()

,Data,Dollar,Oil Brent,Seconds
0,2020-01-01,62.0,66.0,1577836800
1,2020-01-09,62.0,65.0,1578528000
2,2020-01-10,61.0,65.0,1578614400
3,2020-01-11,61.0,65.0,1578700800
4,2020-01-14,61.0,64.0,1578960000


In [2]:
#деление на обучающую и тестовую выборки
#df = df.groupby('int_memory')['battery_power'].mean().reset_index()

x = df.Seconds[100:220].to_numpy()
y = df.Dollar[100:220].to_numpy()

#список индексов, отсортированных по возрастанию x (battery_power)
sorted_indices = np.argsort(x)
x = x[sorted_indices]
y = y[sorted_indices]

# % на выборках
oby4 = round(0.8*(220-100))

x_train = x[:oby4]
y_train = y[:oby4]
x_test = x[oby4:]
y_test = y[oby4:]


print('Обучающая выборка:')
print(x_train)
print(y_train)
print('\nТестовая выборка:')
print(x_test)
print(y_test)

Обучающая выборка:
[1591142400 1591228800 1591315200 1591401600 1591660800 1591747200
 1591833600 1591920000 1592265600 1592352000 1592438400 1592524800
 1592611200 1592870400 1592956800 1593043200 1593129600 1593216000
 1593475200 1593561600 1593648000 1593734400 1593820800 1594080000
 1594166400 1594252800 1594339200 1594425600 1594684800 1594771200
 1594857600 1594944000 1595030400 1595289600 1595376000 1595462400
 1595548800 1595635200 1595894400 1595980800 1596067200 1596153600
 1596240000 1596499200 1596585600 1596672000 1596758400 1596844800
 1597104000 1597190400 1597276800 1597363200 1597449600 1597708800
 1597795200 1597881600 1597968000 1598054400 1598313600 1598400000
 1598486400 1598572800 1598659200 1598918400 1599004800 1599091200
 1599177600 1599264000 1599523200 1599609600 1599696000 1599782400
 1599868800 1600128000 1600214400 1600300800 1600387200 1600473600
 1600732800 1600819200 1600905600 1600992000 1601078400 1601337600
 1601424000 1601510400 1601596800 160168320

In [3]:
#MHK

def func(x, c0, c1, c2): #функция для аппроксимации
    return c0 + c1 * x + c2 * x**2

popt, _ = curve_fit(func, x_train, y_train)
c0, c1, c2 = popt

# Построение аппроксимации и графика данных
y_education = func(x_train, c0, c1, c2)
fig = go.Figure()

# обучающие данные
fig.add_trace(go.Scatter(
    x = x_train,
    y = y_train,
    mode = 'lines',
    name = 'Обучающие данные',
    line=dict(color = '#00BFFF', width = 2))
)

# МНК
fig.add_trace(go.Scatter(
    x = x_train,
    y = y_education,
    mode = 'lines',
    name = 'МНК',
    line = dict(color = '#FF8C00', shape = 'spline', width = 2))
)

fig.update_xaxes(title_text="Время (сек.)")
fig.update_yaxes(title_text="Курс доллара (руб.)")
fig.update_layout(title_text='График "Доллар на 2020 год', title_x =0.5)
fig.show()

In [4]:
#MHK

def func(x, c0, c1, c2): #функция для аппроксимации
    return c0 + c1 * x + c2 * x**2

popt, _ = curve_fit(func, x_test, y_test)
c0, c1, c2 = popt

# Построение аппроксимации и графика данных
y_education = func(x_test, c0, c1, c2)
fig = go.Figure()

# тестовые данные
fig.add_trace(go.Scatter(
    x = x_test,
    y = y_test,
    mode = 'lines',
    name = 'Тестовые данные',
    line=dict(color = '#00BFFF', width = 2))
)

# МНК
fig.add_trace(go.Scatter(
    x = x_test,
    y = y_education,
    mode = 'lines',
    name = 'МНК',
    line = dict(color = '#FF8C00', shape = 'spline', width = 2))
)

fig.update_xaxes(title_text="Время (сек.)")
fig.update_yaxes(title_text="Курс доллара (руб.)")
fig.update_layout(title_text='График "Доллар на 2020 год', title_x =0.5)
fig.show()

In [6]:
#KNn

k = 2  # соседи
model = KNeighborsClassifier(n_neighbors=k)
model.fit(x_train.reshape(-1,1), y_train)
y_pred = model.predict(x_train.reshape(-1,1))
# обучающие данные
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = x_train,
    y = y_train,
    mode = 'lines',
    name = 'Обучающие данные',
    line=dict(color = '#4682B4', width = 2))
)

# МНК
fig.add_trace(go.Scatter(
    x = x_train,
    y = y_pred,
    mode = 'lines',
    name = 'KNn',
    line = dict(color = 'red', width = 2))
)
fig.update_xaxes(title_text="Время (сек.)")
fig.update_yaxes(title_text="Курс доллара (руб.)")
fig.update_layout(title_text='График "Доллар на 2020 год', title_x =0.5)
fig.show()

In [7]:
#KNn

k = 2  # соседи
model = KNeighborsClassifier(n_neighbors=k)
model.fit(x_test.reshape(-1,1), y_test)
y_pred = model.predict(x_test.reshape(-1,1))
# тестовые данные
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = x_test,
    y = y_test,
    mode = 'lines',
    name = 'Тестовые данные',
    line=dict(color = '#4682B4', width = 2))
)

# МНК
fig.add_trace(go.Scatter(
    x = x_test,
    y = y_pred,
    mode = 'lines',
    name = 'KNn',
    line = dict(color = 'red', width = 2))
)
fig.update_xaxes(title_text="Время (сек.)")
fig.update_yaxes(title_text="Курс доллара (руб.)")
fig.update_layout(title_text='График "Доллар на 2020 год', title_x =0.5)
fig.show()

In [8]:
#ARIMA

y_pred_arima=[]
y_train_save = []
for i in y_train:
    y_train_save.append(i) #заполняем уже имеющимися

p = 1  # порядок авторегрессии
d = 1 # порядок интеграции
q = 0 # порядок скользящего среднего
for i in range(len(x_test)):
    model = sm.tsa.arima.ARIMA(y_train_save, order=(p, d, q)).fit()
    y_pred_arima.append(model.forecast()[0])
    y_train_save.append(y_train[i])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x = x,
    y = y_train_save,
    mode = 'lines',
    name = 'Значения',
    line=dict(color = '#4682B4', width = 2))
)

# МНК
fig.add_trace(go.Scatter(
    x = x_test,
    y = y_pred_arima,
    mode = 'lines',
    name = 'ARIMA',
    line = dict(color = 'red', width = 2))
)
fig.update_xaxes(title_text="Время (сек.)")
fig.update_yaxes(title_text="Курс доллара (руб.)")
fig.update_layout(title_text='График "Доллар на 2020 год', title_x =0.5)
fig.show()